# Example 8: Segmented Spectral Fitting

In some advanced cases, a time series may not have a single, uniform scaling behavior across all frequencies. It might, for example, behave like random noise (β ≈ 0) at high frequencies (short timescales) but show strong persistence (β > 1) at low frequencies (long timescales). This indicates a change in the underlying processes governing the system at different scales.

The `waterSpec` package provides a **segmented fitting** feature to automatically detect a single breakpoint (or "knee") in the power spectrum and fit two separate spectral slopes (β₁ and β₂).

This example demonstrates how to use this feature.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.waterSpec.workflow import run_analysis
from src.waterSpec.plotting import plot_spectrum
from fbm import FBM # You may need to install this: pip install fbm

## 1. Generating Synthetic Data with a Spectral Break

To test the segmented fitting, we need to create a time series with a known spectral break. A common way to do this is to combine two signals with different fractal properties:

1.  **Fractional Brownian Motion (fBm):** This will give us a persistent signal with β > 1 at low frequencies.
2.  **White Noise:** This will give us a non-persistent signal with β = 0 at high frequencies.

We will generate an fBm signal and then add white noise to it. The point where the power of the noise signal starts to dominate the power of the fBm signal will create the breakpoint.

In [ ]:
# Generate a time vector (irregularly sampled)
np.random.seed(42)
n_points = 500
time = np.sort(np.random.uniform(0, n_points, n_points))

# Generate a persistent signal (fBm) with Hurst=0.8 (beta=2*H+1 = 2.6, but theory is complex)
# Let's aim for a theoretical beta around 1.8 for the persistent part.
hurst = 0.9
f = FBM(n=n_points-1, hurst=hurst, length=n_points, method='daviesharte')
fbm_signal = f.fbm()

# Generate white noise
noise = np.random.normal(0, 15, n_points) # Noise std dev is 15

# Combine the signals
combined_signal = fbm_signal + noise

# Create a DataFrame
df = pd.DataFrame({
    'timestamp': pd.to_datetime(time, unit='D'),
    'value': combined_signal
})

# Save to a temporary file
file_path = 'segmented_data.csv'
df.to_csv(file_path, index=False)

# Plot the generated time series
plt.figure(figsize=(12, 6))
plt.plot(df['timestamp'], df['value'], marker='.', linestyle='-')
plt.title('Synthetic Time Series with Spectral Break')
plt.xlabel('Time')
plt.ylabel('Value')
plt.grid(True)
plt.show()

## 2. Running the Analysis with `analysis_type='segmented'`

To enable segmented fitting, we simply set the `analysis_type` parameter in `run_analysis` to `'segmented'`.

In [ ]:
results = run_analysis(
    file_path=file_path,
    time_col='timestamp',
    data_col='value',
    param_name='Synthetic Signal',
    analysis_type='segmented', # <-- Key parameter
    do_plot=True,
    output_path='segmented_fit_plot.png'
)

print("Segmented Fit Results:")
print(f"Breakpoint Frequency: {results.get('breakpoint'):.4f}")
print(f"Beta 1 (Low Frequency): {results.get('beta1'):.2f}")
print(f"Beta 2 (High Frequency): {results.get('beta2'):.2f}")

## 3. Interpreting the Results

Let's analyze the output:

- **Beta 1 (Low Frequency):** The value should be significantly greater than 1, reflecting the long-range persistence of the underlying fBm signal. This is the 'memory' of the system.
- **Beta 2 (High Frequency):** The value should be close to 0, reflecting the dominance of white noise at shorter timescales.
- **Breakpoint Frequency:** This is the frequency at which the scaling behavior changes. It represents the point where the process transitions from being dominated by long-term memory to being dominated by short-term randomness.

The plot generated by `do_plot=True` will visually confirm this, showing two distinct lines fitted to the power spectrum, with a clear "knee" at the breakpoint.

In [ ]:
# Clean up the temporary file
import os
os.remove(file_path)